In [301]:
import scipy 
import numpy as np
import pandas as pd


# spam

In [302]:
data=np.load("./data/spam-data-hw3.npz")
data.keys()

KeysView(NpzFile './data/spam-data-hw3.npz' with keys: training_data, training_labels, test_data)

In [303]:
data['training_data'].shape

(4171, 32)

In [304]:
data['training_labels'].shape

(4171,)

In [305]:
training_data=data['training_data']
training_label=data['training_labels']


In [306]:
training_label

array([1, 1, 1, ..., 0, 0, 0], shape=(4171,))

In [307]:
unique_labels,counts=np.unique(training_label,return_counts=True)
print(unique_labels)
print(counts)
prior_probs=counts/len(training_label)

[0 1]
[2971 1200]


In [308]:
def normalize_features(features):
    # L2 normalize to every vector
    norms=np.linalg.norm(features,axis=1,keepdims=True) # get the L2 norm for each vector
    norms[norms==0]=1 # Take care of the zero case
    normalized_features=features/norms # Normalization

    return normalized_features



In [309]:
normalized_data=normalize_features(training_data)
print(normalized_data.shape)


(4171, 32)


In [310]:
normalized_data

array([[0.        , 0.        , 0.        , ..., 0.54433105, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.5547002 , 0.        ,
        0.2773501 ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.81511151, 0.232889  ,
        0.        ]], shape=(4171, 32))

In [311]:
# MLE fit Gaussian Distribution
from statistics import covariance


def fit_gaussian(training_data,training_labels):
    n_classes=len(np.unique(training_labels))
    n_features=training_data.shape[1]

    means=np.zeros((n_classes,n_features))
    covariances=np.zeros((n_classes,n_features,n_features))

    for class_label in range(n_classes):
        class_mask=(training_labels==class_label)
        class_data=training_data[class_mask]

        means[class_label]=np.mean(class_data,axis=0)
        centered_data=class_data-means[class_label]


        covariances[class_label]=np.cov(centered_data.T)


    return means,covariances


In [312]:
means,covariances=fit_gaussian(training_data=normalized_data,training_labels=training_label)


In [313]:
means.shape

(2, 32)

In [314]:
covariances.shape

(2, 32, 32)

In [315]:
def quadratic_func(x,mean,cov,prior_prob):
    # quadratic function as probability
    centered_x=x-mean
    try:
        inv_cov=np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        inv_cov=np.linalg.pinv(cov)
    quadratic_term=-0.5*centered_x.T@inv_cov@centered_x

    det_cov=np.linalg.det(cov)
    if det_cov<=0:
        det_term=0

    else:
        det_term=-0.5*np.log(det_cov)
    
    prior_term=np.log(prior_prob)

    return quadratic_term+det_term+prior_term

In [316]:
def predict_all_simples(test_data,means,covariances,prior_probs):
    predictions=[]

    for i in range(len(test_data)):
        sample=test_data[i]

        class_scores=[]
        for k in range(len(means)):
            score=quadratic_func(sample,means[k],covariances[k],prior_probs[k])
            class_scores.append(score)

        predicted_class=np.argmax(class_scores)
        predictions.append(predicted_class)

    return np.array(predictions)
    

In [317]:
test_data=data['test_data']

In [318]:
normalized_test_data=normalize_features(test_data)

In [319]:
predictions=predict_all_simples(test_data=normalized_test_data,means=means,covariances=covariances,prior_probs=prior_probs)

In [320]:
predictions

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,

In [321]:
unique_labels,counts=np.unique(predictions,return_counts=True)
print(unique_labels)
print(counts)

[0 1]
[593 407]


# mnist

In [322]:
data=np.load("./data/mnist-data-hw3.npz")
data.keys()

KeysView(NpzFile './data/mnist-data-hw3.npz' with keys: training_data, training_labels, test_data)

In [323]:
data['training_data'].shape

(60000, 1, 28, 28)

In [324]:
data['training_data'].reshape(60000,28*28)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(60000, 784), dtype=uint8)

In [325]:
training_data=data['training_data']
training_label=data['training_labels']


In [326]:
training_data=training_data.reshape(60000,28*28)

In [327]:
training_label

array([3, 7, 1, ..., 3, 7, 1], shape=(60000,), dtype=uint8)

In [328]:
unique_labels,counts=np.unique(training_label,return_counts=True)
print(unique_labels)
print(counts)
prior_probs=counts/len(training_label)

[0 1 2 3 4 5 6 7 8 9]
[5923 6742 5993 6162 5862 5428 5870 6213 5851 5956]


In [329]:
def normalize_features(features):
    # L2 normalize to every vector
    norms=np.linalg.norm(features,axis=1,keepdims=True) # get the L2 norm for each vector
    norms[norms==0]=1 # Take care of the zero case
    normalized_features=features/norms # Normalization

    return normalized_features



In [330]:
normalized_data=normalize_features(training_data)
print(normalized_data.shape)


(60000, 784)


In [331]:
normalized_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(60000, 784))

In [332]:
# MLE fit Gaussian Distribution
from statistics import covariance


def fit_gaussian(training_data,training_labels):
    n_classes=len(np.unique(training_labels))
    n_features=training_data.shape[1]

    means=np.zeros((n_classes,n_features))
    covariances=np.zeros((n_classes,n_features,n_features))

    for class_label in range(n_classes):
        class_mask=(training_labels==class_label)
        class_data=training_data[class_mask]

        means[class_label]=np.mean(class_data,axis=0)
        centered_data=class_data-means[class_label]


        covariances[class_label]=np.cov(centered_data.T)


    return means,covariances


In [333]:
means,covariances=fit_gaussian(training_data=normalized_data,training_labels=training_label)


In [334]:
means.shape

(10, 784)

In [335]:
covariances.shape

(10, 784, 784)

In [336]:
def quadratic_func(x,mean,cov,prior_prob):
    # quadratic function as probability
    centered_x=x-mean
    try:
        inv_cov=np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        inv_cov=np.linalg.pinv(cov)
    quadratic_term=-0.5*centered_x.T@inv_cov@centered_x

    det_cov=np.linalg.det(cov)
    if det_cov<=0:
        det_term=0

    else:
        det_term=-0.5*np.log(det_cov)
    
    prior_term=np.log(prior_prob)

    return quadratic_term+det_term+prior_term

In [337]:
def predict_all_simples(test_data,means,covariances,prior_probs):
    predictions=[]

    for i in range(len(test_data)):
        sample=test_data[i]

        class_scores=[]
        for k in range(len(means)):
            score=quadratic_func(sample,means[k],covariances[k],prior_probs[k])
            class_scores.append(score)

        predicted_class=np.argmax(class_scores)
        predictions.append(predicted_class)

    return np.array(predictions)
    

In [338]:
test_data=data['test_data']

In [339]:
test_data=test_data.reshape(test_data.shape[0],-1)

In [340]:
normalized_test_data=normalize_features(test_data)

In [341]:
predictions=predict_all_simples(test_data=normalized_test_data,means=means,covariances=covariances,prior_probs=prior_probs)

KeyboardInterrupt: 

In [ ]:
predictions

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,

In [ ]:
unique_labels,counts=np.unique(predictions,return_counts=True)
print(unique_labels)
print(counts)

[0 1]
[593 407]


In [342]:
import matplotlib.pyplot as plt

In [346]:
np.save('cov_matrix_digit_0.npy', cov_matrix)